In [2]:
import sys
import os

# Add the root directory to the Python environment
root_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(root_dir)
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# Read the CSV file
df = pd.read_csv("../data/stats/stats_20250710_170427.csv")
df.head()

,block_number,top_coins,num_nodes,num_edges,max_degree,mean_degree,treewidth
0,22820000,5000,5000.0,15846.0,9276.0,6.34,32
1,22820000,5000,NaN,NaN,NaN,NaN,32
2,22820000,10000,10000.0,27062.0,18996.0,5.41,38
3,22820000,10000,NaN,NaN,NaN,NaN,38
4,22820000,15000,15000.0,37596.0,28852.0,5.01,41


In [4]:
df = pd.read_csv("../data/updates/updates_22816000.csv.gz")
df.head()

,pool_id,pool_status,block_number,token0,token1,event,version,feeTier,sqrtPrice,tick
0,0x4746e745b75278a0847beea79d621cc722c657d7,updated,22816094,"{'decimals': '9', 'id': '0x3f70f50ab9c1bb821b7...","{'decimals': '18', 'id': '0xc02aaa39b223fe8d0a...",swap,v2,NaN,NaN,NaN
1,0xcfb26df385d790aa7e417394ec1196a3bd56aa8c,updated,22816070,"{'decimals': '18', 'id': '0x774eaf7a5347162876...","{'decimals': '18', 'id': '0xc02aaa39b223fe8d0a...",swap,v2,NaN,NaN,NaN
2,0xb71b4e954f132f3770d9bde0d1d5eb50c32147ef,updated,22816003,"{'decimals': '9', 'id': '0x5417994ae69e6ccb283...","{'decimals': '18', 'id': '0xc02aaa39b223fe8d0a...",swap,v2,NaN,NaN,NaN
3,0x4ae55d9c5fd8f991379eeb3852fb20cab14f74b8,updated,22816078,"{'decimals': '9', 'id': '0x9b032a259dbce03f733...","{'decimals': '18', 'id': '0xc02aaa39b223fe8d0a...",swap,v2,NaN,NaN,NaN
4,0x360f6c06034dda01cc2356268f60c15a8937542c,updated,22816001,"{'decimals': '18', 'id': '0x76887cb94cf29ec539...","{'decimals': '18', 'id': '0xc02aaa39b223fe8d0a...",swap,v2,NaN,NaN,NaN


In [ ]:
from_block = 22816000
to_block = 22821000

In [10]:
df_top_tokens = pd.read_csv("../data/top_tokens_22816000.csv")
df_top_tokens.head()


,token_id,token_symbol
0,0xd286511359a0b7f3d77d96a4fce06a9226a24353,unknown
1,0xf7498c98789957f4ee53b3e37ff5b7ef8a6cfc7b,0xDEV
2,0x3cb48aeb3d1abadc23d2d8a6894b3a68338381c2,PALAI
3,0x9cc32f63850a3d9a64d613a4fa7b1c7d8993c054,CRYPT
4,0x5483dc6abda5f094865120b2d251b5744fc2ecb5,TPAD


In [72]:
from src.stats import safe_literal_eval


jump_size = 5000
results_rows = []
# for top_coins in range(jump_size, 100000+jump_size, jump_size):
top_coins = 100000
# Prepare set of top tokens for fast lookup
top_token_set = set(df_top_tokens['token_id'].head(top_coins).tolist())

# Track visited tokens and pools for this top_coins group
visited_tokens = set()

# Counters
update_price = 0
new_token = 0
new_pool = 0

# For each update file in data/updates
updates_dir = "../data/updates"
# if not os.path.exists(updates_dir):
#     print(f"Directory {updates_dir} does not exist.")
#     continue

import glob
update_files = sorted(glob.glob(os.path.join(updates_dir, "*.csv.gz")))
for update_file in update_files:
    df_updates = pd.read_csv(update_file)
    df_updates['token0'] = df_updates['token0'].apply(safe_literal_eval)
    df_updates['token1'] = df_updates['token1'].apply(safe_literal_eval)

    # Each file contains updates for 100 blocks.
    # So, we should group by block_number and process each block within the file.
    if 'block_number' in df_updates.columns:

        for block, group in df_updates.groupby('block_number'):
            block_update_price = 0
            block_new_token = 0
            block_new_pool = 0
            block_visited_tokens = set()
            block_visited_pools = set()
            for idx, row in group.iterrows():
                token0 = row['token0']['id']
                token1 = row['token1']['id']
                pool_id = row['pool_id']

                token0_in = token0 in top_token_set
                token1_in = token1 in top_token_set

                # New pool logic (per block)
                if row['pool_status'] == 'new' and pool_id not in block_visited_pools:
                    block_new_pool += 1
                    block_visited_pools.add(pool_id)

                if block == 22816004:
                    print(token0_in, token1_in)
                    print(token0, token1)

                # New token logic (per block)
                if not token0_in or not token1_in:
                    if not token0_in and token0 not in block_visited_tokens:
                        block_new_token += 1
                        block_visited_tokens.add(token0)
                    if not token1_in and token1 not in block_visited_tokens:
                        block_new_token += 1
                        block_visited_tokens.add(token1)
                else:
                    # Both tokens are in top_token_set
                    block_update_price += 1

            # Create a row for this block_number and top_coins
            row_dict = {
                'block_number': block,
                'top_coins': top_coins,
                'update_price': block_update_price,
                'new_token': block_new_token,
                'new_pool': block_new_pool
            }
            results_rows.append(row_dict)
            # print(f"Processed {block} for top_coins={top_coins}")
    # print(f"Processed {update_file} for top_coins={top_coins}")


df_results = pd.DataFrame(results_rows)
df_results.head()


True True
0x1223334444a7466fbf985b14e1f4edaf3883bca6 0xb10cc888cb2cce7036f4c7ecad8a57da16161338
True True
0x19640000000ba88d36206beb10d0e86011c8d08c 0xb10cc888cb2cce7036f4c7ecad8a57da16161338
False True
0x0000000000000000000000000000000000000000 0x19640000000ba88d36206beb10d0e86011c8d08c


,block_number,top_coins,update_price,new_token,new_pool
0,22816000,100000,6,1,0
1,22816001,100000,11,1,0
2,22816002,100000,4,1,0
3,22816003,100000,9,0,0
4,22816004,100000,2,1,0


In [ ]:
df_results.to_csv("../data/updates/updates_stats.csv", index=False)


In [5]:
df = pd.read_csv("../data/update_stats.csv.gz")
df.head(2000)

,block_number,top_coins,update_price,new_token,new_pool
0,22816000,100000,6,0,0
1,22816001,100000,11,0,0
2,22816002,100000,4,0,0
3,22816003,100000,9,0,0
4,22816004,100000,2,0,0
...,...,...,...,...,...
458,22840995,100000,6,3,0
459,22840996,100000,2,0,0
460,22840997,100000,7,2,0
461,22840998,100000,9,0,0


In [75]:
top_token_set = set(df_top_tokens['token_id'].head(100000).tolist())

In [1]:
df = pd.read_csv("../data/updates/updates_22816000.csv.gz")
df = df[df['block_number'] == 22816004]
# df = df[df['event'] == 'swap']
df['token0'] = df['token0'].apply(safe_literal_eval)
df['token1'] = df['token1'].apply(safe_literal_eval)
df['token0_id'] = df['token0'].apply(lambda x: x.get('id') if isinstance(x, dict) else None)
df['token1_id'] = df['token1'].apply(lambda x: x.get('id') if isinstance(x, dict) else None)

df['token0_in_top'] = df['token0_id'].apply(lambda x: x in top_token_set)
df['token1_in_top'] = df['token1_id'].apply(lambda x: x in top_token_set)
print(df[['token0_id', 'token0_in_top', 'token1_id', 'token1_in_top']])

df.head()

NameError: name 'pd' is not defined